In [55]:
import requests
from pprint import pprint
from bs4 import BeautifulSoup
import pandas as pd
import time
import schedule


In [2]:
url = 'https://www.cbr-xml-daily.ru/daily_json.js'
response = requests.get(url)

In [3]:
# response = requests.get('https://www.cbr-xml-daily.ru/daily.xml')


currencies = response.json()
# pprint(currencies)

pprint(currencies['Valute']['CZK']['Name'])

'Чешских крон'


In [4]:
url = 'https://nplus1.ru/news/2021/10/11/econobel2021'
response = requests.get(url)
page = BeautifulSoup(response.text, 'html.parser')
print(page.title)
print(page.title.text)

<title>Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей</title>
Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей


In [5]:
print(page.find('h1').text)
print(page.find('time').text)

Премию Нобеля по экономике присудили за исследования экономики труда и причинно-следственных связей

13:04
11 Окт. 2021



In [6]:
# print(page.find('div', class_='body').text) 
# print(page.find('div', class_='js-mediator-article').text)
print(page.find('div', class_='body js-mediator-article').text)









Премия Шведского национального банка по экономическим наукам памяти Альфреда Нобеля за 2021 год присуждена Дэвиду Карду (David Card) за его вклад в эмпирические исследования экономики рынка труда, а также Джошуа Энгристу (Joshua Angrist) и Гвидо Имбенсу (Guido Imbens) за их вклад в методологию анализа причинно-следственных связей. Прямая трансляция церемонии объявления лауреатов шла на официальном сайте Нобелевской премии.









Слева направо: Дэвид Карт, Джошуа Энгрист и Гвидо Имбенс.
Niklas Elmehed © Nobel Prize Outreach



Поделиться

















Премия по экономике, в отличие от Нобелевских премий по физиологии и медицине, физике, химии и литературе, а также премии мира, была учреждена не самим Альфредом Нобелем, а Банком Швеции в 1968 году. Ее размер равен размеру остальных премий.Лауреаты этого года награждены за их исследования причинно-следственных связей в области социальных наук. В частности, Дэвид Кард из университета Калифорнии в Беркли начиная с 1990-х годов

In [7]:
def get_wiki_header(wiki_url):
    response = requests.get(wiki_url)
    header = BeautifulSoup(response.text, 'html.parser')
    return header.find('h1').text

In [8]:
url = 'https://en.wikipedia.org/wiki/List_of_programming_languages'
response = requests.get(url) # Делаем GET-запрос к ресурсу
page = BeautifulSoup(response.text, 'html.parser') # Создаём объект BeautifulSoup
links = page.find_all('a')
len(links)

print([link.text for link in links[0:10]])

['', 'Jump to navigation', 'Jump to search', 'Programming languagelists', 'Alphabetical', 'Categorical', 'Chronological', 'Generational', 'v', 't']


### ЗАПРОС К API ИЗ КОДА


In [9]:
token = '4b3ce8894b3ce8894b3ce8892e4b47e5f944b3c4b3ce8892ac8de372367576846ac59d8'
url = 'https://api.vk.com/method/users.get'
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'}
response = requests.get(url, params=params)
pprint(response.json())

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


In [10]:
user = response.json()['response'][0]
print(user['first_name'])

Павел


In [11]:
ids = ','.join(map(str, range(1,4)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bday', 'access_token': token, 'lang': 'ru'}
pprint(requests.get(url, params=params).json())

{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


In [ ]:
ids = ','.join(map(str, range(1, 501)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': token, 'lang': 'ru'}
sex_response =  requests.get(url, params=params).json()['response']
# pprint(sex_response)
df = pd.DataFrame(sex_response)
df[(df['sex'] == 1) | (df['sex'] == 2)]['sex'].value_counts(normalize=True)



In [33]:
url = 'https://api.vk.com/method/groups.getMembers'
params = {'group_id': 'vk', 'v': 5.95, 'access_token': token}
response = requests.get(url, params=params)
data = response.json()
print(data['response']['items'][:20])

[5, 6, 19, 34, 47, 54, 79, 177, 193, 198, 212, 219, 239, 243, 254, 345, 404, 406, 407, 467]


In [36]:
url = 'https://api.vk.com/method/groups.getMembers'
count = 1000
offset = 0
user_ids = []
max_count = data['response']['count']
while offset < 10000:
    print('Выгружаю {} пользователей с offset = {}'.format(count, offset))   
    params = {'group_id': 'vk', 'v': 5.95, 'count': count, 'offset': offset, 'access_token': token}
    response = requests.get(url, params = params)
    data = response.json()
    user_ids += data['response']['items']
    offset += count
    print('Ожидаю 0.5 секунды...')
    time.sleep(0.5)
print('Цикл завершен, offset =', offset)


Выгружаю 1000 пользователей с offset = 0
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 1000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 2000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 3000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 4000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 5000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 6000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 7000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 8000
Ожидаю 0.5 секунды...
Выгружаю 1000 пользователей с offset = 9000
Ожидаю 0.5 секунды...
Цикл завершен, offset = 10000


In [ ]:
url = 'https://api.vk.com/method/wall.get'
params = {'domain': 'vk', 'filter': 'owner', 'count': 1000, 'offset': 0, 'access_token': token, 'v': 5.95}
response = requests.get(url, params = params)
pprint(response.json())

In [47]:
response.json()['response']['items'][0]

{'id': 1270492,
 'from_id': -22822305,
 'owner_id': -22822305,
 'date': 1642084584,
 'marked_as_ads': 0,
 'post_type': 'post',
 'text': '[https://vk.com/checkback#snowfall|«Снегопад подарков»] осыпал участников призами во второй раз 🏆\n\nЗаходите в мини-приложение: если вам улыбнулась удача, увидите свой уникальный номер в блоке «Победители». Мы заботимся о личных границах и безопасности участников: в списке нет фамилий, фото и ссылок на профили. Но будем рады, если призёры поделятся своими победами прямо в комментариях!\n\nСо всеми победителями свяжемся в личных сообщениях в течение суток и расскажем, как получить подарки. Не волнуйтесь, если вам ещё не написали: мы уже рядом 😉\n\nНо на этом вихрь сюрпризов не заканчивается — продолжайте получать моментальные призы в «Снегопаде подарков» до 15 января. А ещё на следующей неделе объявим результаты розыгрыша от банка ПСБ. Если собирали специальные снежинки, следите за новостями!',
 'attachments': [{'type': 'photo',
   'photo': {'album_id

In [48]:
stats = {}
count_post = 0
for record in response.json()['response']['items'][:]:
    title = record['text'][:30]
    if title: 
        stats[title] = [record['comments']['count'], record['reposts']['count'], record['date']]
        count_post += 1
    if count_post < 10:
        continue
    else: 
        break
pprint(stats)

{'[https://vk.com/checkback#snow': [1218, 1690, 1642084584],
 'А если взглянуть в целом, то к': [275, 259, 1640094467],
 'ВКонтакте всегда есть что посм': [197, 92, 1640017040],
 'Запускаем собственное шоу в пе': [95, 1369, 1640940328],
 'Каким был ваш 2021-й ВКонтакте': [601, 9770, 1640174402],
 'Мы приготовили для вас целую г': [173, 651, 1640712137],
 'Объявляем итоги первого розыгр': [665, 898, 1640871560],
 'Одни называют всё, что происхо': [127, 275, 1639491162],
 'Приближаем праздники вместе — ': [7043, 685, 1639590427],
 'Эти каникулы будут особенными ': [130, 156, 1640171709]}


### Автоматический запуск - скрипты

In [49]:
!pip install schedule

In [ ]:
def task():
    print('Hello! I am a task!')
    return

schedule.every(1).minutes.do(task)

while True: 
    schedule.run_pending()
    time.sleep(1)

In [59]:
def number_exponential(num):
   return num**2

number_list = [2,3,4,5]

print(list(map(number_exponential, number_list)))

[4, 9, 16, 25]


In [60]:
iter_example = ['test', 'test2', 'test3']
for idx, val in enumerate(iter_example):
   print(idx)
   print(val)

0
test
1
test2
2
test3


In [62]:
dict_comprehension = {key:val for key, val in enumerate('sample')}
print(dict_comprehension)

{0: 's', 1: 'a', 2: 'm', 3: 'p', 4: 'l', 5: 'e'}


In [93]:
def sample(*args):
    print(args)

sample('time', 1, True)

#Пример **kwargs
def sample(**kwargs):
    print(kwargs)

sample(a = 'time', b = 1)

('time', 1, True)
{'a': 'time', 'b': 1}


In [96]:
def star_triangle(r):
   for x in range(r):
      print(' '*(r-x-1)+'*'*(2*x+1))

star_triangle(7)

      *
     ***
    *****
   *******
  *********
 ***********
*************


In [109]:
# Функция чтения данных
def read_data():
    data = open('/Users/maxmercury/Downloads/war_peace_processed.txt', 'rt', encoding='utf-8').read()
    data = data.split('\n')
    return data


# Функция подсчёта частоты употребления слова в тексте
def freq(target_word):  
    new_dict = dict() 
    for i in data: # СТРОКА С ОШИБКОЙ
        if i not in new_dict:
            new_dict[i] = 1
        else:
            new_dict[i] = new_dict.get(i) + 1
    return new_dict.get(target_word, 0)


# Вызов функций
read_data()
freq('война')

0